In [ ]:
import torch
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, random_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import os

In [ ]:
# Load original .pth
raw_dataset = torch.load("EEG-ImageNet_1.pth", weights_only=False)

# Filter by granularity
filtered = [d for d in raw_dataset["dataset"] ]

# Label map
label_set = sorted(list(set(d["label"] for d in filtered)))
label_to_idx = {label: i for i, label in enumerate(label_set)}

# Allocate arrays
X = np.stack([d["eeg_data"].numpy().astype(np.float32) for d in filtered])
Y = np.array([label_to_idx[d["label"]] for d in filtered], dtype=np.int32)

print("Final shapes:", X.shape, Y.shape)  # Example: (num_samples, channels, time)

# Save as .npz
os.makedirs("EEGNetNPY", exist_ok=True)
np.savez("EEGNetNPY/eeg_cleaned_all.npz", X=X, Y=Y)
print("Clean file saved to eeg_cleaned_all.npz")


In [ ]:
# Training loop with early stopping
best_acc = 0.0
patience = 5
no_improve = 0
epochs = 50

criterion = LabelSmoothingCrossEntropy(smoothing=0.1)  # Using label smoothing

for epoch in range(1, epochs + 1):
    train_loss, train_acc = train(model, train_loader)
    test_loss, test_acc = evaluate(model, test_loader)
    
    print(f"📚 Epoch {epoch}/{epochs}:")
    print(f"   Train Loss: {train_loss:.4f} | Acc: {train_acc*100:.2f}%")
    print(f"   Test Loss:  {test_loss:.4f} | Acc: {test_acc*100:.2f}%")
    
    # Save best model
    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), 'best_model.pth')
        no_improve = 0
        print("💾 Saved new best model!")
    else:
        no_improve += 1
    
    # Early stopping
    if no_improve >= patience:
        print(f"⏹ Early stopping at epoch {epoch}")
        break

# Load best model
model.load_state_dict(torch.load('best_model.pth'))
final_test_loss, final_test_acc = evaluate(model, test_loader)
print(f"\n🎯 Final Test Accuracy: {final_test_acc*100:.2f}%")

In [59]:
import torch.optim as optim
from tqdm import tqdm

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [60]:
def train(model, loader):
    model.train()
    running_loss, correct = 0.0, 0
    total = 0
    for inputs, labels in tqdm(loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        preds = outputs.argmax(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total

def evaluate(model, loader):
    model.eval()
    running_loss, correct = 0.0, 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            preds = outputs.argmax(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return running_loss / total, correct / total


In [61]:
epochs = 20
for epoch in range(1, epochs + 1):
    train_loss, train_acc = train(model, train_loader)
    test_loss, test_acc = evaluate(model, test_loader)
    print(f"📚 Epoch {epoch}: Train Acc = {train_acc:.4f}, Test Acc = {test_acc:.4f}")


100%|██████████| 25/25 [00:09<00:00,  2.67it/s]


📚 Epoch 1: Train Acc = 0.1919, Test Acc = 0.2100


100%|██████████| 25/25 [00:09<00:00,  2.76it/s]


📚 Epoch 2: Train Acc = 0.2681, Test Acc = 0.2475


100%|██████████| 25/25 [00:11<00:00,  2.27it/s]


...

📚 Epoch 18: Train Acc = 0.4838, Test Acc = 0.3525


100%|██████████| 25/25 [00:10<00:00,  2.41it/s]


📚 Epoch 19: Train Acc = 0.4894, Test Acc = 0.3650


100%|██████████| 25/25 [00:09<00:00,  2.53it/s]


📚 Epoch 20: Train Acc = 0.5050, Test Acc = 0.3600
